这里PAGA环境比较老，本地不好配置，启动docker，

```
docker run -it \
    -v /home/huang/RCode/scrna_tools/dynverse_reproduce/ti_paga:/ti/workspace \
    --entrypoint /bin/bash \
    dynverse/ti_paga:v0.9.9.05
```

附加VSCode，安装python，jupyter插件,调试后续的代码

数据输入和参数

In [ ]:
import dynclipy


# 参数部分
tmp_dir = "tmp"
input_h5 = f"{tmp_dir}/input.h5"
output_h5 = f"{tmp_dir}/output.h5"
# args = f"--dataset {input_h5} --output output_h5"
args = ["--dataset", input_h5, "--output", output_h5]

# 方法描述定义部分
definition_location = "definition.yml" 

task = dynclipy.main(args, definition_location) # 内部其实是调用了R的代码
task

轨迹推断方法执行

In [ ]:
# avoid errors due to no $DISPLAY environment variable available when running sc.pl.paga
import matplotlib
matplotlib.use('Agg')

import pandas as pd
import numpy as np
import h5py
import json

# import scanpy.api as sc
import scanpy as sc
import anndata
import numba
import warnings

import time
checkpoints = {}

#   ____________________________________________________________________________
#   Load data                                                               ####
#  NOTE: 导入数据，从task转化为表达矩阵，可以学习一下，后续可能是需要这里的task wrapper与AnnData融合


counts = task["counts"]

parameters = task["parameters"]

start_id = task["priors"]["start_id"]
if isinstance(start_id, list):
  start_id = start_id[0]

if "groups_id" in task["priors"]:
  groups_id = task["priors"]['groups_id']
else:
  groups_id = None

# create dataset
if groups_id is not None:
  obs = pd.DataFrame(groups_id)
  obs.index = groups_id["cell_id"] # 细胞ID
  obs["louvain"] = obs["group_id"].astype("category") # 细胞聚类结果, 后续在AnnData数据上关于细胞聚类的操作都是基于lovain的
  adata = anndata.AnnData(counts)
  adata.obs = obs
else:
  adata = anndata.AnnData(counts)

checkpoints["method_afterpreproc"] = time.time()

#   ____________________________________________________________________________
#   Basic preprocessing                                                     ####
#  NOTE: 基础的预处理，加了很多日志

# normalisation & filtering
if counts.shape[1] < 100 and parameters["filter_features"]:
  print("You have less than 100 features, but the filter_features parameter is true. This will likely result in an error. Disable filter_features to avoid this")

if parameters["filter_features"]:
  n_top_genes = min(2000, counts.shape[1])
  sc.pp.recipe_zheng17(adata, n_top_genes=n_top_genes)

# precalculating some dimensionality reductions
sc.tl.pca(adata, n_comps=parameters["n_comps"])
with warnings.catch_warnings():
  warnings.simplefilter('ignore', numba.errors.NumbaDeprecationWarning)
  sc.pp.neighbors(adata, n_neighbors=parameters["n_neighbors"])

# denoise the graph by recomputing it in the first few diffusion components
if parameters["n_dcs"] != 0:
  sc.tl.diffmap(adata, n_comps=parameters["n_dcs"])

#   ____________________________________________________________________________
#   Cluster, infer trajectory, infer pseudotime, compute dimension reduction ###
#  NOTE: 聚类、轨迹推断、计算降维

# add grouping if not provided
if groups_id is None:
  sc.tl.louvain(adata, resolution=parameters["resolution"])

# run paga
sc.tl.paga(adata)

# compute a layout for the paga graph
# - this simply uses a Fruchterman-Reingold layout, a tree layout or any other
#   popular graph layout is also possible
# - to obtain a clean visual representation, one can discard low-confidence edges
#   using the parameter threshold
sc.pl.paga(adata, threshold=0.01, layout='fr', show=False)

# run dpt for pseudotime information that is overlayed with paga
adata.uns['iroot'] = np.where(adata.obs.index == start_id)[0][0]
if parameters["n_dcs"] == 0:
  sc.tl.diffmap(adata)
sc.tl.dpt(adata, n_dcs = min(adata.obsm['X_diffmap'].shape[1], 10))

# run umap for a dimension-reduced embedding, use the positions of the paga
# graph to initialize this embedding
if parameters["embedding_type"] == 'umap':
  sc.tl.umap(adata, init_pos='paga')
  dimred_name = 'X_umap'
else:
  sc.tl.draw_graph(adata, init_pos='paga')
  dimred_name = "X_draw_graph_" + parameters["embedding_type"]

checkpoints["method_aftermethod"] = time.time()

In [ ]:
import pprint
# 主要结果
print("louvain".center(50, '='))
pprint.pprint(adata.obs["louvain"].head(5))
print("dpt pseudotime".center(50, '='))
pprint.pprint(adata.obs["dpt_pseudotime"].head(5))
print("paga connectivities".center(50, '='))
pprint.pprint(adata.uns["paga"]["connectivities"].A)

=====================louvain======================
1_iN1_C01    3
1_iN1_C02    3
1_iN1_C03    3
1_iN1_C04    0
1_iN1_C05    0
Name: louvain, dtype: category
Categories (6, object): [0, 1, 2, 3, 4, 5]
==================dpt pseudotime==================
1_iN1_C01    0.000000
1_iN1_C02    0.024606
1_iN1_C03    0.045618
1_iN1_C04    0.810991
1_iN1_C05    0.616293
Name: dpt_pseudotime, dtype: float32
===============paga connectivities================
array([[0.          , 0.0013689134, 0.0405245574, 0.0877420846,
        0.0169298271, 0.0044600082],
       [0.0013689134, 0.          , 0.1311088621, 0.0265885105,
        0.0451462056, 0.0178400329],
       [0.0405245574, 0.1311088621, 0.          , 0.2268733422,
        0.235850005 , 0.2718298109],
       [0.0877420846, 0.0265885105, 0.2268733422, 0.          ,
        0.6686196457, 0.          ],
       [0.0169298271, 0.0451462056, 0.235850005 , 0.6686196457,
        0.          , 0.          ],
       [0.0044600082, 0.0178400329, 0.27182981

结果转换

In [44]:
#   ____________________________________________________________________________
#   Process & save output                                                   ####
#  NOTE: 至此完成了的轨迹推断任务的操作，后续即是结果的封装

# grouping
grouping = pd.DataFrame({"cell_id": adata.obs.index, "group_id": adata.obs.louvain})

# milestone network(n_louvain, n_louvain)
# 细胞聚类间的距离矩阵，宽数据转化为常数
milestone_network = pd.DataFrame(
  np.triu(adata.uns["paga"]["connectivities"].todense(), k = 0),
  index=adata.obs.louvain.cat.categories,
  columns=adata.obs.louvain.cat.categories
).stack().reset_index()
milestone_network.columns = ["from", "to", "length"]
milestone_network = milestone_network.query("length >= " + str(parameters["connectivity_cutoff"])).reset_index(drop=True) # 距离矩阵设置阈值阶段
milestone_network["directed"] = False

# dimred， (n_obs, 2)
dimred = pd.DataFrame([x for x in adata.obsm[dimred_name].T]).T
dimred.columns = ["comp_" + str(i+1) for i in range(dimred.shape[1])]
dimred["cell_id"] = adata.obs.index

# branch progressions: the scaled dpt_pseudotime within every cluster，(n_obs, 3)
branch_progressions = adata.obs
# 每个细胞聚类为一个branch
branch_progressions["dpt_pseudotime"] = branch_progressions["dpt_pseudotime"].replace([np.inf, -np.inf], 1) # replace unreachable pseudotime with maximal pseudotime # 排除异常值
branch_progressions["percentage"] = branch_progressions.groupby("louvain")["dpt_pseudotime"].apply(lambda x: (x-x.min())/(x.max() - x.min())).fillna(0.5) # 最大最小归一化
branch_progressions["cell_id"] = adata.obs.index
branch_progressions["branch_id"] = branch_progressions["louvain"].astype(np.str)
branch_progressions = branch_progressions[["cell_id", "branch_id", "percentage"]] # 只保留这三列

# branches，(n_obs, 3):
# - length = difference between max and min dpt_pseudotime within every cluster # 长度为伪时间的跨度
# - directed = not yet correctly inferred  # 方向至今还没有正确推断出来
branches = adata.obs.groupby("louvain").apply(lambda x: x["dpt_pseudotime"].max() - x["dpt_pseudotime"].min()).reset_index()
branches.columns = ["branch_id", "length"]
branches["branch_id"] = branches["branch_id"].astype(np.str)
branches["directed"] = True

# branch network: determine order of from and to based on difference in average pseudotime
# 添加带方向的网络结构
branch_network = milestone_network[["from", "to"]]
average_pseudotime = adata.obs.groupby("louvain")["dpt_pseudotime"].mean()
for i, (branch_from, branch_to) in enumerate(zip(branch_network["from"], branch_network["to"])):
  if average_pseudotime[branch_from] > average_pseudotime[branch_to]:
    branch_network.at[i, "to"] = branch_from
    branch_network.at[i, "from"] = branch_to

结果保存

In [ ]:
# 在监视窗口实时查看R中的对象, 使用ro.globalenv[dataset.id]
dataset = dynclipy.wrap_data(cell_ids = adata.obs.index)
dataset.add_branch_trajectory(
  grouping = grouping,
  branch_progressions = branch_progressions,
  branches = branches,
  branch_network = branch_network
) # 看看Python里的add_branch_trajectory是如何工作的
dataset.add_dimred(dimred = dimred)
dataset.add_timings(checkpoints)

# 额外添加的add之前的数据,方便放到R里调试
before_add = dict(
    # wrap_add
    cell_ids = adata.obs.index,
    # add_branch_trajectory(主要调试目标)
    grouping = grouping,
    branch_progressions = branch_progressions,
    branches = branches,
    branch_network = branch_network,
    # add_dimred
    dimred = dimred,
    # checkpoints
    checkpoints = checkpoints

)


import rpy2.robjects as ro
ro.globalenv["before_add"] = ro.ListVector([[name, x] for name, x in before_add.items()]) # 待添加的内容转换到R变量里
ro.r(f"{dataset.id}$before_add = before_add") # 调用R修改

dataset.write_output(task["output"])

# dataset

/usr/local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:62: UserWarning: Error while trying to convert the column "from". Fall back to string conversion. The error is: <class 'numpy.str_'>
  % (name, str(e)))
/usr/local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:62: UserWarning: Error while trying to convert the column "to". Fall back to string conversion. The error is: <class 'numpy.str_'>
  % (name, str(e)))
